# Investigate why there's ~80% disconnections for Autoimmune Direct Attack

Load data and imports

In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Jun 16 17:59:10 2023

@author: uconn
"""


################################
# Imports
################################

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


################################
# Load Data
################################
#%%
data_file_path = '../../data/as_metadata/metadata.csv'
data = pd.read_csv(data_file_path, delimiter='\t')


Let's take a look at the size of the avoid lists

In [4]:
# Analyze the size of the avoid list
unique_avoid_lists = data.loc[:, 'avoid_list'].unique()
unique_avoid_lists

array(['[133275]', '[8359]', '[132770]', '[48276]', '[4230]', '[62214]',
       '[33651]', '[19429]', '[6768]', '[209]', '[12912]', '[20804]',
       '[53181]', '[262764]', '[9002]', '[9583]', '[9121]', '[48858]',
       '[9498]', '[24400]', '[9050]', '[16246]', '[19257]', '[174]',
       '[50473]', '[62]', '[51346]', '[30844]', '[272006]', '[61135]',
       '[13335]', '[3549]', '[3356]', '[42020]', '[24348]', '[8447]',
       '[3255]', '[306]', '[41364]', '[1820]', '[33185]', '[26827]',
       '[4261]', '[26801]', '[328748]', '[16814]', '[9299]', '[17762]',
       '[13576]', '[5384]', '[20473]', '[2914]', '[9830]', '[376]',
       '[1239]', '[286]', '[1101]', '[3216]', '[37965]'], dtype=object)

The avoid list of each one is just a single AS ... this is true for every pecentage value I checked ...

> This must be true because the Direct AutoImmune attack targets a single provider of the origin. Therefore, the avoid list would have at most one ASN in it for this attack for the given attacking subprefix.

Some ASes don't have the avoid list ASN in their path to the origin, most likely because the origin is a multihomed AS; which means the adopting ASes that have a successful connection to the origin are connected via the path of another provider
of the multihomed origin.

From here we need to check what fraction of edge ASes are multihomed
> Fraction of edge ASes that are multihomed: 56.56%

## Continue 
Convert the `as_path` and `avoid_list` to list types for easier manipulation later

In [6]:
################################
# Preprocess
################################
#%%

# Convert list data values (which are default interpreted as strings) to list data types
data['as_path'] = data['as_path'].apply(eval)
data['avoid_list'] = data['avoid_list'].apply(eval)

In [13]:
################################
# Analyze
################################
#%%

trial = 1
percentage = 0.1

# Count the disconnected in a single trial (adopting)
adopting_ases_trials = data.loc[data['adoption_setting'] == 'ROV V4 Lite K2', :]
disconnected_adopting_ases_trials = adopting_ases_trials.loc[adopting_ases_trials['outcome'] == 'Outcomes.DISCONNECTED', :]
successful_adopting_ases_trials = adopting_ases_trials.loc[adopting_ases_trials['outcome'] == 'Outcomes.VICTIM_SUCCESS', :]


sinlge_trial_data = disconnected_adopting_ases_trials.loc[(disconnected_adopting_ases_trials['trial'] == trial) & (disconnected_adopting_ases_trials['percentage'] == percentage), :]
successful_single_trial_data = successful_adopting_ases_trials.loc[(successful_adopting_ases_trials['trial'] == trial) & (successful_adopting_ases_trials['percentage'] == percentage), :]

ten_percent_trials = adopting_ases_trials.loc[adopting_ases_trials['percentage'] == 0.1, :]

# Check the fraction of disconnected / all_ases (should be close to 80% according to the phenomenon we're investigating)
print(len(disconnected_adopting_ases_trials)/len(adopting_ases_trials))
print(len(successful_adopting_ases_trials)/len(adopting_ases_trials))

0.8465923826011346
0.15334258628226285


This is consistent with the results seen so far

Lets check if all the disconnected is due to a blackhole?

In [14]:
#%%
# Check the contents of their RIBs
## Do they have a blackhole?
## Do they have a path to the origin without a blackhole?
single_trial_prefix_outcome = sinlge_trial_data.loc[sinlge_trial_data['prefix_for_outcome'] == sinlge_trial_data['local_rib_prefix'], :]
disconnected_ases_with_blackhole = sum(single_trial_prefix_outcome['blackhole'] == True)
# Fraction of ASes that are disconnected due to blackhole -- It should be all of them -- it is ...
print(disconnected_ases_with_blackhole/len(single_trial_prefix_outcome['asn'].unique()))

1.0


All disconnections are indeed due to blackholes, as expected.

In [15]:
# Lets check the RIB of those who are successfully connected. How is it possible?
# Did they not install a blackhole? For what reason?
successful_single_trial_data_prefix_outcome = successful_single_trial_data.loc[successful_single_trial_data['prefix_for_outcome'] == successful_single_trial_data['local_rib_prefix'], :]
print(len(successful_single_trial_data_prefix_outcome))

0


It seems that those who are successfully connected do not have a path to the prefix choosen for the outcome.
That's of course true, because this is a direct autoimmune attack (i.e. there is no actual announcement)

In [16]:
# Check if any of the successfully connected have a member of the avoid list in there path
avoid_list_item_in_success_path = successful_single_trial_data.apply(lambda row: any(asn in row.as_path for asn in row.avoid_list), axis=1)
print(any(avoid_list_item_in_success_path))

False


In [17]:
successful_single_trial_data.head()

,trial,percentage,propagation_round,asn,adoption_setting,outcome,prefix_for_outcome,local_rib_prefix,as_path,relationship,blackhole,avoid_list
82464,1,0.1,0,48528,ROV V4 Lite K2,Outcomes.VICTIM_SUCCESS,1.2.2.0/24,1.2.0.0/16,[48528],Relationships.ORIGIN,False,[8359]
82467,1,0.1,0,209122,ROV V4 Lite K2,Outcomes.VICTIM_SUCCESS,1.2.2.0/24,1.2.0.0/16,"[209122, 3356, 3216, 48528]",Relationships.PROVIDERS,False,[8359]
82468,1,0.1,0,209122,ROV V4 Lite K2,Outcomes.VICTIM_SUCCESS,1.2.2.0/24,1.2.1.0/24,"[209122, 3356, 3216, 48528]",Relationships.PROVIDERS,True,[8359]
82515,1,0.1,0,33491,ROV V4 Lite K2,Outcomes.VICTIM_SUCCESS,1.2.2.0/24,1.2.0.0/16,"[33491, 3356, 3216, 48528]",Relationships.PROVIDERS,False,[8359]
82516,1,0.1,0,33491,ROV V4 Lite K2,Outcomes.VICTIM_SUCCESS,1.2.2.0/24,1.2.1.0/24,"[33491, 3356, 3216, 48528]",Relationships.PROVIDERS,True,[8359]


We can see that all the successfully connected do not have a path with the avoid list ASN in their path to the origin.